# Dit RAG ? 🤖

## Récupération de la base de données vectorielle

On va déjà récupérer la base de données vectorielle des publis DIRAG

In [ ]:
import os

from s3fs import S3FileSystem

fs = S3FileSystem(
    client_kwargs={"endpoint_url": f"https://{os.environ['AWS_S3_ENDPOINT']}"},
    key=os.environ["AWS_ACCESS_KEY_ID"],
    secret=os.environ["AWS_SECRET_ACCESS_KEY"],
)

In [ ]:
s3_path = "projet-llm-insee-open-data/data/chroma_database/experiment/dirag/"
local_path = "data/chroma_db_checkpoint/"

fs.download(
    rpath=s3_path,
    lpath=local_path,
    recursive=True,
)

On _load_ la base dans `Python` grâce à `langchain`

In [ ]:
from src.db_building.build_database import load_vector_database_from_local

db = load_vector_database_from_local(
    persist_directory="data/chroma_db_checkpoint/", embedding_model="OrdalieTech/Solon-embeddings-large-0.1"
)

On vérifie juste que ça fonctionne sur une requête simple

In [ ]:
db.similarity_search("quel est le secteur dominant en guadeloupe?")

On transforme en `retriever` langchain

In [ ]:
from src.db_building import load_retriever

retriever, vectorstore = load_retriever(
    vectorstore=db,
    retriever_params={"search_type": "similarity", "search_kwargs": {"k": 10}},
)

In [ ]:
retrieved_docs = retriever.invoke("subventions aux entreprises")

## Chain RAG vllm

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from src.utils import create_prompt_from_instructions, format_docs

system_instructions = """
Tu es un assistant spécialisé dans la statistique publique.
Tu réponds à des questions concernant les données de l'Insee,
l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte,
réponds de manière argumentée à la question posée.

La réponse doit être développée et citer l'ensemble de ses sources
(titre et url de la publication) qui sont référencées à la fin.
Cite notamment l'url d'origine de la publication, dans un format markdown.
Un lien hypertext vers l'url de la source serait apprécié.

Cite 10 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.

Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
"""

question_instructions = """
Voici la question à laquelle tu dois répondre :
Question: {question}

Réponse:
"""

In [ ]:
from openai import OpenAI

# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "https://projet-llm-insee-open-data-vllm.user.lab.sspcloud.fr/v1/"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


def statbot_from_vllm_api(
    question,
    retriever,
    client,
    system_instructions=system_instructions,
    question_instructions=question_instructions,
    model="mistralai/Mistral-Small-24B-Instruct-2501",
):
    prompt = create_prompt_from_instructions(system_instructions, question_instructions)
    context = format_docs(retriever.invoke(question))

    question_with_context = prompt.format(question=question, context=context)

    chat_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_instructions},
            {"role": "user", "content": question_with_context},
        ],
    )
    return chat_response, chat_response.choices[0].message.content

In [ ]:
from IPython.display import Markdown, display

_, answer = statbot_from_vllm_api(
    "Quelles sont les subventions spécifiques aux DOMS et leur effet sur le tissu productif ?", retriever, client
)
display(Markdown(answer.replace("   ", "")))

In [ ]:
_, answer = statbot_from_vllm_api("La Guadeloupe est-elle défavorisée par rapport à la Métropole ?", retriever, client)
display(Markdown(answer.replace("   ", "")))

In [ ]:
_, answer = statbot_from_vllm_api("y a quoi à manger à la cantine ce soir?", retriever, client)
display(Markdown(answer.replace("   ", "")))

## Chain RAG locale

In [ ]:
from langchain_community.llms import VLLM

from src.config import MODEL_TO_ARGS

LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"

llm = VLLM(model=LLM_MODEL, **MODEL_TO_ARGS.get(LLM_MODEL, {}))

In [ ]:
system_instructions = """
Tu es un assistant spécialisé dans la statistique publique.
Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.

La réponse doit être développée et citer ses sources (titre et url de la publication) qui sont référencées à la fin.
Cite notamment l'url d'origine de la publication, dans un format markdown.

Cite 5 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.

Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
"""

question_instructions = """
Voici la question à laquelle tu dois répondre :
Question: {question}

Réponse:
"""

prompt = create_prompt_from_instructions(system_instructions, question_instructions)

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [ ]:
from IPython.display import Markdown, display

answer = rag_chain.invoke("Quel est le niveau de vie en Guadeloupe?")

display(Markdown(answer.replace("   ", "")))